In [1]:
import os, sys, re
from functools import partial

import pandas as pd
import numpy as np

from bibtexparser.latexenc import string_to_latex

sys.path.insert(1, os.path.join(sys.path[0], '..'))

import database
from snowballing.operations import load_work_map_all_years, work_to_bibtex, reload
from snowballing.operations import match_bibtex_to_work
from snowballing.approaches import get_approaches, name, wlatex_name, wcitea
reload()

all_approaches = get_approaches()
script = [(a, m) for a, m in all_approaches if not m["binary"]]
binary = [(a, m) for a, m in all_approaches if m["binary"]]
len(script)

27

In [2]:
{y if isinstance(y,str) else y.value for a, m in all_approaches for y in m["query"]}

{'Command',
 'Function Calls',
 'Functions',
 'Interoperable Format',
 'PROV',
 'Proprietary',
 'Query',
 'Web'}

In [3]:
{a.display 
 for a, m in all_approaches
 for y in m["query"]
 if (y if isinstance(y,str) else y.value)=='Graph File'}

set()

In [4]:
from database.groups import REPRODUCIBILITY
from database.groups import YES, NO

def select(element, field):
    try:
        index = field.index(element)
        element = field[index]
        if hasattr(element, "_star") and element._star is not None:
            return element
        return YES
    except ValueError:
        return NO
    
def find(elements, abrevs, field):
    for element, abrev in zip(elements, abrevs):
        try:
            index = field.index(element)
            if (hasattr(element, "_star") and element._star is not None) or abrev is None:
                yield element
            yield abrev
        except ValueError:
            pass

writer = pd.ExcelWriter('output\management2.xlsx')
dfs = {}

for mode_name, mode in [("script", script), ("binary", binary)]:

    management_header = ["Approach", "Storage", "Distribution", "Versioning"]
    management_extra = ["Approach"] + ["Management"] * 3
    management_data = [
        [
            name(approach),
            ", ".join(map(str, meta["storage"])) or NO,
            ", ".join(map(str, meta["distribution"])) or NO,
            str(meta["evolution"]),
           # select(REPRODUCIBILITY, meta["supports"]),
        ]
        for approach, meta in mode
    ]

    management = pd.DataFrame.from_records(
        [management_header] +
        [list(map(str, elements)) for elements in management_data],
        columns=management_header,
    )
    management = management.set_index("Approach")
    
    analysis_header = ["Approach", "Visualization", "Summarization", "Query", "Comparison"]
    analysis_extra = ["Approach"] + ["Analysis"] * 4
    analysis_data = [
        [
            name(approach),
            ", ".join(map(str, meta["visualization"])) or NO,
            ", ".join(map(str, meta["summarization"])) or NO,
            ", ".join(map(str, meta["query"])) or NO,
            str(meta["diff"])
        ]
        for approach, meta in mode
    ]

    analysis = pd.DataFrame.from_records(
        [analysis_header] +
        [list(map(str, elements)) for elements in analysis_data],
        columns=analysis_header,
    )
    analysis = analysis.set_index("Approach")


    df = pd.concat([management, analysis], axis=1)
    df.index.name = None
    df.columns = management_extra[1:] + analysis_extra[1:]
    df.to_excel(writer,mode_name)
    dfs[mode_name] = df
    
writer.save()

In [5]:
dfs["script"]

,Management,Management,Management,Analysis,Analysis,Analysis,Analysis
Approach,Storage,Distribution,Versioning,Visualization,Summarization,Query,Comparison
Astro-WISE,Relational Database (Oracle),✗,Trial Sequence,Tree,✗,"Query (SQL), Functions, Web",[Provenance]
Becker and Chambers,Proprietary,"Proprietary, Source",✗,Process View (Statements),✗,"Functions (Command Provenance, Data Provenance)",[]
"Bochner, Gude, and Schreiber",✗,Web (PReServ),✗,✗,✗,"Query (XQuery, XPath), Web (SOAP messages)",[]
CPL,"Relational Database (MySQL, PostgreSQL), Graph...",Proprietary (JSON),Trial Identification,✗,✗,"Query (SPARQL, SQL, Web), Functions",[]
CXXR,Memory,✗,✗,✗,✗,"Functions (Command Provenance, Data Provenance)",[]
Datatrack,"VCS, Proprietary (CSV)","Version Control System, Proprietary (CSV)",Trial Sequence,Data View,Clustering (Combines all data accesses in nodes),✗,[VCS]
ES3,NoSQL (XML Server),"Graph File (GraphML, GraphViz)",✗,"Combined View (GraphML, GraphViz)",✗,Query (XQuery),[]
ESSW,"Relational Database (MySQL), Content Database",Graph File (GraphViz),*ID,Combined View (GraphViz),✗,"Web, Query (SQL)",[]
IncPy,Content Database,Content Database,✗,✗,✗,✗,"[Provenance, cache]"
